In [ ]:
betaco <- c(-6,1.0,0.005)
n <- seq(30,630,200)
no.rep <- 1000
n.max <- 100
tol <- 1e-9
meanMLE <- matrix(NA,length(n),3)

allMLE <- array(NA,c(no.rep,3,length(n)))
for(n.loop in 1:length(n)){
    MLE <- matrix(NA,no.rep,3)

    for(i in 1:no.rep){
        set.seed(i)
        gpa <- rnorm(n[n.loop],3.1,0.3)
        gre <- rnorm(n[n.loop],580,80) 
        x.beta <- betaco[1]+betaco[2]*gpa+betaco[3]*gre
        pi.admit <- exp(x.beta)/(1+exp(x.beta))
        Y <- c()
        for (j in 1:n[n.loop]){
            Y[j] <- sample(c(0,1),1,c(1-pi.admit[j],pi.admit[j]),replace=F)        
        }    
        X <- cbind(rep(1,length(Y)),gpa,gre)
        
        betacoef <- c(0,0,0)
        pil <- exp(X %*% betacoef)/(1+exp(X %*% betacoef))
        grad <- t(X) %*% (Y-pil)
        hess <- -t(X) %*% diag(c(pil*(1-pil)),length(Y)) %*% X
        n.iter <- 0
        while ((max(abs(grad))> tol) & (n.iter < n.max) ) {
            betacoef <- betacoef - solve(hess) %*% grad
            pil <- exp(X %*% betacoef)/(1+exp(X %*% betacoef))
            grad <- t(X)%*%(Y-pil)
            hess <- -t(X) %*% diag(c(pil*(1-pil)),length(Y)) %*% X
            n.iter <- n.iter + 1
        }
        if (n.iter== n.max) {
            cat("fail to converge\n")
        } else {
            MLE[i,] <- betacoef
        }

    }
    meanMLE[n.loop,] <- colSums(MLE)/no.rep

    allMLE[,,n.loop] <- MLE

}
